In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import PIL
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.image as mpimg
from tensorflow.keras.models import Sequential

In [ ]:
import pathlib
data_path = pathlib.Path("/kaggle/input/mushrooms3/mushrooms")

image_count = len(list(data_path.glob('*/*.jpg')))
print(image_count)

In [ ]:
nrows = 3
ncols = 3

pos = 0

for subfolder in os.listdir(data_path):
    
    image_file = os.listdir(os.path.join(data_path, subfolder))[0]
    
    fig = plt.gcf()
    fig.set_size_inches(ncols * 4, nrows * 4)

    pos += 1
    sp = plt.subplot(nrows, ncols, pos)

    cur_image = mpimg.imread(os.path.join(data_path, subfolder, image_file))
    plt.imshow(cur_image)
    plt.title(subfolder)
    plt.axis('Off')

In [ ]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import glob
image_list = []
i=0
for filepath in glob.glob('/kaggle/input/mushrooms3/mushrooms/*/*.jpg', recursive=True):
    mushroom_class = filepath.split("/")[-2]
    image_list.append((filepath, mushroom_class))
    
data = pd.DataFrame(image_list, columns = ['filepath', 'name'])
print('data shape: {}'.format(data.shape))

In [ ]:
mushroom_classes = data['name'].unique()
mushroom_classes

In [ ]:
data['name'].value_counts().plot(kind='bar')
plt.title('Mushroom class distribution')
plt.xlabel('Mushroom Name')
plt.ylabel('Count')
plt.show()

In [ ]:
batch_size = 128
img_height = 224
img_width = 224

In [ ]:
train_split = 0.7
val_split = 0.2
test_split = 0.1

# Eğitim 
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels="inferred",
    label_mode="int",
    validation_split=val_split + test_split,
    subset="training",
    seed=101,
    shuffle=True,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# Doğrulama 
remaining_ds = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels="inferred",
    label_mode="int",
    validation_split=val_split + test_split,
    subset="validation",
    seed=101,
    shuffle=True,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# Doğrulama ve test veri setini val_split oranında doğrulama ve test olarak ayırma
val_size = int(len(remaining_ds) * (val_split / (val_split + test_split)))
test_size = len(remaining_ds) - val_size
val_ds = remaining_ds.take(val_size)
test_ds = remaining_ds.skip(val_size)

# Bilgilendirme mesajları
print("Eğitim veri seti boyutu:", len(train_ds))
print("Doğrulama veri seti boyutu:", len(val_ds))
print("Test veri seti boyutu:", len(test_ds))


In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 20))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_dsTr = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_dsV = val_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_dsTe = test_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_dsTr))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
from tensorflow.keras import Model,regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input,Flatten,Dropout,BatchNormalization,GlobalAveragePooling2D,Reshape

num_classes = len(class_names)

resnet_model = ResNet50(weights="imagenet",include_top=False,pooling='avg',classes=9,input_shape=(224, 224, 3))
last_layer = resnet_model.get_layer("avg_pool")
resnet_layers = Model(inputs=resnet_model.inputs, outputs=last_layer.output)

for layer in resnet_model.layers:
    layer.trainable = False

for layer in resnet_model.layers[-10:]:
    layer.trainable = True

    
x = resnet_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x) 
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l1(0.001))(x)  # Dropout yerine x kullanın

model = Model(resnet_model.input, output)


x = Dense(512,activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = Dense(256,activation='relu')(x)
x = layers.Dropout(0.4)(x)
x = Dense(64,activation='relu')(x)
x = layers.Dropout(0.25)(x)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("Model_ResNet50.keras", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq="epoch")
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,
                              mode='max', cooldown=2, patience=2, min_lr=0)

In [ ]:
epochs = 30
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[checkpoint, early, reduce_lr],
)

In [ ]:
model.save("model.h5")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(47)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training Accuracy')
plt.title('Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training Loss')
plt.title('Validation Loss')
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(test_ds)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_pred = model.predict(test_ds)
y_pred_labels = np.argmax(y_pred, axis=1)  

# 
y_true = np.concatenate([y for x, y in test_ds], axis=0)

# 
print("Classification Report:")
print(classification_report(y_true, y_pred_labels))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
conf_mat = confusion_matrix(y_true, y_pred_labels)

num_classes = len(class_names)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, cmap='Blues', fmt='g', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Tahmin Edilen Sınıf')
plt.ylabel('Gerçek Sınıf')
plt.title('Karışıklık Matrisi')
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.show()